In [ ]:
w = 12
l = 16
xts = Any[];

### Expected Threat Function

In [ ]:
function expected_threat(p_scoring, p_shot, p_move, transition_matrix)
    
    """Solves the expected threat equation with dynamic programming.

    Parameters
    ----------
    p_scoring : (np.ndarray, shape(M, N)):
        Probability of scoring at each grid cell, when shooting from that cell.
    p_shot : (np.ndarray, shape(M,N)):
        For each grid cell, the probability of choosing to shoot from there.
    p_move : (np.ndarray, shape(M,N)):
        For each grid cell, the probability of choosing to move from there.
    transition_matrix : (np.ndarray, shape(M*N,M*N)):
        When moving, the probability of moving to each of the other zones.
    """
    
    xT = zeros(w, l)
    eps = 1e-5
    gs = p_scoring .* p_shot
    it = 0
    diff = 1 

    while length(findall(diff-> diff .> eps, diff)) > 0 ###
        total_payoff = zeros(w, l)

        for y in 0:w-1
            for x in 0:l-1
                ##for each zone, calculate the probability of moving to all other zones times the xT at those zones
                for q in 0:w-1
                    for z in 0:l-1
                        total_payoff[y+1, x+1] += transition_matrix[(l*y + x)+1, (l*q + z)+1] * xT[q+1, z+1]
                    end
                end
            end
        end
        
        push!(xts, xT)
        newxT = gs + (p_move .* total_payoff)
        diff = newxT - xT
        xT = newxT
        it+=1
        
    end
    println("Total iterations to convergence: ", it)
    return xT
end

### Getting the matrices

In [ ]:
using Plots
using DataFrames
using CSV
using StatsBase

df = DataFrame(CSV.File("xt_pre_data.csv"));

In [ ]:
df[1:6, [:team_id, :player_name, :start_x, :start_y, :end_x, :end_y, :type_name, :result_name]]

In [ ]:
names(df)

In [ ]:
function get_bins(values, n_bins, max_lim)
    idx = Int8.(values.÷(max_lim/n_bins) .+ 1)
    idx[findall(idx -> idx>n_bins, idx)] .= n_bins
    idx
end

In [ ]:
function get_matrix(df)
    df[!, :start_x_bin] = get_bins(df[!, :start_x], l, 105)
    df[!, :start_y_bin] = get_bins(df[!, :start_y], w, 68)
    
    gdf = combine(groupby(df, [:start_x_bin, :start_y_bin]), nrow => :count);
    
    matrix = zeros(w, l)
    for a in eachrow(gdf)
        matrix[a.start_y_bin, a.start_x_bin] = a.count
    end
    matrix
end

In [ ]:
c_shot_matrix = get_matrix(df[df["type_name"] .== "shot", :])
c_goal_matrix = get_matrix(df[(df["type_name"] .== "shot") .& (df["result_name"] .== "success"), :])
c_move_matrix = get_matrix(df[in.(df["type_name"], Ref(["dribble", "pass", "cross"])), :])

c_total_matrix = c_move_matrix + c_shot_matrix
c_move_matrix = c_move_matrix ./ c_total_matrix
c_score_matrix = c_goal_matrix ./ c_shot_matrix; replace!(c_score_matrix, NaN=>0)
c_shot_matrix = c_shot_matrix ./ c_total_matrix;

c_move_matrix = reverse(c_move_matrix, dims=1)
c_score_matrix = reverse(c_score_matrix, dims=1)
c_shot_matrix = reverse(c_shot_matrix, dims=1);

### Transition Matrix

In [ ]:
df[!, :start_x_bin] = get_bins(df[!, :start_x], l, 105)
df[!, :start_y_bin] = get_bins(df[!, :start_y], w, 68)

df[!, :end_x_bin] = get_bins(df[!, :end_x], l, 105)
df[!, :end_y_bin] = get_bins(df[!, :end_y], w, 68);

In [ ]:
get_flat_idx(xs, ys, l) = Int16.(((l .* ys) .+ xs) .- l)

function get_transition_matrix(df)
    """Calculate the transition_matrix
    #Arguments: 
      The dataframe of move_actions
    #Returns:
      transition_matrix (shape=(w*l, w*l))   
    """
	trans_matrix = zeros(w*l, w*l)
	for i in 1:w*l
	    vc2 = countmap(df[(df["start_flat_idx"] .== i) , :]["end_flat_idx"])
	    for (key, value) in vc2
	        trans_matrix[i, key] = value/vals[i]
	    end
	end
	trans_matrix
end

df[!, "start_flat_idx"] = get_flat_idx(df["start_x_bin"], df["start_y_bin"], l)
df[!, "end_flat_idx"] = get_flat_idx(df["end_x_bin"], df["end_y_bin"], l)

vals = get_matrix(df[in.(df["type_name"], Ref(["dribble", "pass", "cross"])), :])
vals = vec(reverse(vals, dims=1)');

succ_move_actions = df[in.(df["type_name"], Ref(["dribble", "pass", "cross"])) .& (df["result_name"] .== "success"), :]
c_trans_matrix = get_transition_matrix(succ_move_actions);

In [ ]:
cxT = expected_threat(c_score_matrix, c_shot_matrix, c_move_matrix, c_trans_matrix);